In [6]:
import sys, os



%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")

import os
os.environ['R_HOME'] = 'D:/Program Files/R-4.5.0' #@TODO include in some setup file

from simulation_engine.scenarios.iv.binary_iv import BinaryIV
from simulation_engine.scenarios.conf.binary_conf import BinaryConf
from simulation_engine.scenarios.iv.continuous_iv import ContinuousIV
from simulation_engine.scenarios.customDAG.customBinaryDAG import CustomBinaryDAG
from simulation_engine.util.plotting_util import PlottingUtil
from simulation_engine.algorithms.causaloptim import Causaloptim
from simulation_engine.algorithms.zhang_bareinboim import ZhangBareinboim
from simulation_engine.algorithms.apid import Apid
from simulation_engine.algorithms.autobound import AutoBound
from simulation_engine.algorithms.apid_src.src.models.apid import APID

import pandas as pd
pd.set_option('display.max_columns', None)

#disable warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from simulation_engine.algorithms.zaffalonbounds import ZaffalonBounds
from hydra import compose, initialize
from omegaconf import OmegaConf

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
sim = BinaryConf._generate_data(
    n=500
)
dataConf = pd.DataFrame({'Y': sim['Y'], 'X': sim['X']})
dataConf

,Y,X
0,1,1
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
495,1,0
496,1,1
497,1,0
498,1,1


In [43]:
AutoBound.bound(query='ATE', 
                dagstring='X->Y, U -> X, U -> Y' ,
                data=dataConf, 
                observed=['X', 'Y'], 
                unob=['U'], 
                indep='X', 
                dep='Y')

(-0.442, 0.558)

In [60]:
dataMed, ATE_true = CustomBinaryDAG.generateMediation(500)
dataMed
print(ATE_true)

0.162401916993256


In [ ]:
# AutoBound.bound(query='ATE', 
#                 dagstring='X->Y, M->Y, U -> X, U -> Y, U -> M',
#                 data=dataMed, 
#                 observed=['X', 'Y', 'M'], 
#                 unob=['U'], 
#                 indep='X', 
#                 dep='Y')
customdag = CustomBinaryDAG(dataMed,
                            dagstring='X->Y, M->Y, U -> X, U -> Y, U -> M ',
                            unobserved=['U'],
                            indep='X',
                            dep='Y',
                            )


customdag.observed

['M', 'X', 'UB', 'Y']